In [34]:
from pathlib import Path
import json
import plotly.express as px
import pandas as pd
from json.decoder import JSONDecodeError
import logging

target_env = ".env"


logs_dir = Path("../data/logs")

files = list(logs_dir.glob('*.log'))


In [25]:
data = []

for i in files:
    data_in_this_file = i.read_text().splitlines()
    for i in data_in_this_file:
        try:
            if target_env in i:
                data+=data_in_this_file
                break
        except KeyError:
            pass
            #print("key error")
        except JSONDecodeError:# mongo relatedMongoLogger
            print("json decode error")

In [26]:
data[:5]

['{"created": 1609824462.248437, "logger": "job_1609824459_dbad0212caa4431c8ce3a807ac22ab9b", "level": "INFO", "message": "Processing started.", "meta": {"label": "START_PROCESS", "counter_config": {"di": 10, "mcdf": 2, "mctf": 3, "detector": "yolo", "tracker": "kcf", "use_droi": false, "droi": [[0, 0], [1920, 0], [1920, 1080], [0, 1080]], "counting_lines": [{"label": "main_crosswalk", "line": [[562, 842], [1913, 441]]}, {"label": "road1", "line": [[331, 421], [573, 812]]}, {"label": "road2", "line": [[304, 472], [821, 202]]}, {"label": "road3", "line": [[812, 198], [1242, 279]]}], "argfile": "tokyo_crosswalk.env"}}}',
 '{"created": 1609824462.3531783, "logger": "job_1609824459_dbad0212caa4431c8ce3a807ac22ab9b", "level": "INFO", "message": "Object counted.", "meta": {"label": "OBJECT_COUNT", "id": "obj_24d87f3bd0894c17b8b671223b652e87", "type": "car", "counting_line": "road1", "position_first_detected": [560, 726], "position_counted": [560, 726], "counted_at": 1609824462.3531601}}',
 '

In [27]:
rows = []

for a in data:
    try:
        i= json.loads(a)
        if i["level"] == "INFO" and i["meta"]["label"] == "OBJECT_COUNT":
            rows.append(i)
    except TypeError:
        pass

            

In [28]:
rows = [row["meta"] for row in rows]

In [29]:
rows[:4]

[{'label': 'OBJECT_COUNT',
  'id': 'obj_24d87f3bd0894c17b8b671223b652e87',
  'type': 'car',
  'counting_line': 'road1',
  'position_first_detected': [560, 726],
  'position_counted': [560, 726],
  'counted_at': 1609824462.3531601},
 {'label': 'OBJECT_COUNT',
  'id': 'obj_5fff2727ad3e4e1ab5167717ec36d224',
  'type': 'car',
  'counting_line': 'road2',
  'position_first_detected': [608, 363],
  'position_counted': [608, 364],
  'counted_at': 1609824462.353401},
 {'label': 'OBJECT_COUNT',
  'id': 'obj_221111f6c1b64408915103cf36e22407',
  'type': 'car',
  'counting_line': 'main_crosswalk',
  'position_first_detected': [1143, 602],
  'position_counted': [1143, 602],
  'counted_at': 1609824462.3535514},
 {'label': 'OBJECT_COUNT',
  'id': 'obj_c4fefbaf72af41c898984b26e72740e3',
  'type': 'car',
  'counting_line': 'road3',
  'position_first_detected': [1098, 272],
  'position_counted': [1098, 272],
  'counted_at': 1609824462.353717}]

In [30]:
from pymongo import MongoClient
import urllib.parse
import uuid

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)
db = client["traffic"]
mycol = db["traffic_data"]


In [31]:
class MongoLogger:
    def __init__(self):
        self.server_uuid = self.get_server_uuid()

    def log_data(self, data_dict: dict):

        data_dict["computer_id"] = self.server_uuid
        data_dict["arg_file"] = target_env
        #mycol.insert_one(data_dict)

    def log_data_many(self, data_list: [dict]):
        for i in data_list:
            i["computer_id"] = self.server_uuid
            i["arg_file"] = target_env
        mycol.insert_many(data_list)

    def get_server_uuid(self):
        computer_id = str(uuid.UUID(int=uuid.getnode()))
        return computer_id



In [32]:
ml = MongoLogger()

In [33]:
ml.log_data_many(rows)